In [2]:

from src.models.preclassifier import *


In [3]:

# -------------------------
# Demo / main
# -------------------------
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Device: {device} | CPU cores: {multiprocessing.cpu_count()} | RAM: ~24GB reported")
# generate scenes
scenes_train = [ example_scene_generator(num_sensors=12, duration_sec=1.0, seed=i) for i in range(100) ]
scenes_val = [ example_scene_generator(num_sensors=12, duration_sec=1.0, seed=100+i) for i in range(20) ]
# Pretrain: collect encoders dict
model = SensorStructureModel(rates=[200], emb_dim=128, time_bins=32, device=device)

# Pretrain encoders
print("=== Contrastive pretrain encoders ===")
encoders = model.encoders
pretrain_contrastive(encoders, scenes_train, device=device, epochs=2, batch_size=128, lr=1e-3)

num_epochs = 6
# Fine-tune full model
train_ds = SceneDataset(scenes_train)
val_ds = SceneDataset(scenes_val)
print("=== Finetune full model ===")
finetune_full(model, train_ds, val_ds, device=device, epochs=num_epochs, lr=1e-3)
# Save checkpoint
ckpt_path = "checkpoints/model_ckpt.pth"
save_checkpoint(ckpt_path, model, optimizer=None, epoch=num_epochs, extra={'rates': model.rates})
# Evaluate and print
print("=== Evaluation on training set ===")
res_train = evaluate_model(model, train_ds, device=device)
print("=== Evaluation on validation set ===")
res_val = evaluate_model(model, val_ds, device=device)
# Load into fresh model and continue training for comparison
print("=== Reload checkpoint and continue training (compare) ===")
model2 = SensorStructureModel(rates=[200], emb_dim=128, time_bins=32, device=device)
load_checkpoint(ckpt_path, model2, map_location=device)
# continue training few more epochs
finetune_full(model2, train_ds, val_ds, device=device, epochs=3, lr=5e-4)
print("=== Post-resume evaluation on validation set ===")
res_val2 = evaluate_model(model2, val_ds, device=device)
print("Summary: val before resume:", res_val, " val after resume:", res_val2)

# ===== PRE-CLASSIFIER INTEGRATION =====
print("\n" + "="*80)
print("=== PRE-CLASSIFIER: SENSOR TYPE CLASSIFICATION ===")
print("="*80)


Device: cpu | CPU cores: 2 | RAM: ~24GB reported
=== Contrastive pretrain encoders ===
[Pretrain] Pool size: 1487 sensors
[Pretrain] Epoch 1/2 avg_loss=4.618612
[Pretrain] Epoch 2/2 avg_loss=4.248173
=== Finetune full model ===


/workspaces/AIbemyEYE/src/models/preclassifier.py:338: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=use_amp)
/workspaces/AIbemyEYE/src/models/preclassifier.py:368: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=use_amp):


[Finetune] Ep 1/6 train_loss=29.252389 (updated_steps=100)
[Val] node_acc=0.663 edge_prf_mean=0.000 bit_iou=0.151 overflow_f1=0.045


/workspaces/AIbemyEYE/src/models/preclassifier.py:368: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=use_amp):


[Finetune] Ep 2/6 train_loss=24.899688 (updated_steps=100)
[Val] node_acc=0.728 edge_prf_mean=0.000 bit_iou=0.793 overflow_f1=0.045


...

[Finetune] Ep 1/3 train_loss=22.973397 (updated_steps=100)
[Val] node_acc=0.914 edge_prf_mean=0.000 bit_iou=0.553 overflow_f1=0.045


/workspaces/AIbemyEYE/src/models/preclassifier.py:368: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=use_amp):


[Finetune] Ep 2/3 train_loss=22.791952 (updated_steps=100)
[Val] node_acc=0.943 edge_prf_mean=0.000 bit_iou=0.434 overflow_f1=0.045


/workspaces/AIbemyEYE/src/models/preclassifier.py:368: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=use_amp):


[Finetune] Ep 3/3 train_loss=22.739457 (updated_steps=100)
[Val] node_acc=0.949 edge_prf_mean=0.019 bit_iou=0.542 overflow_f1=0.045
=== Post-resume evaluation on validation set ===
[Eval] scenes=20 node_acc=0.9492 edge_f1=0.0194 bit_iou=0.5416 overflow_f1=0.0446
Summary: val before resume: {'node_acc': 0.9091369047619047, 'edge_f1': 0.05416666666666666, 'bit_iou': 0.08482993197278912, 'overflow_f1': 0.04456415822105392}  val after resume: {'node_acc': 0.9491666666666667, 'edge_f1': 0.01944444444444444, 'bit_iou': 0.5415618359814788, 'overflow_f1': 0.04456415822105392}

=== PRE-CLASSIFIER: SENSOR TYPE CLASSIFICATION ===


In [4]:

# Import pre_classifier
try:
    from pre_classifier import SensorPreClassifier
    
    print("\n[Pre-classifier] Initializing...")
    pre_classifier = SensorPreClassifier(use_deep_learning=True, device=device)
    
    # Convert scenes to pre_classifier format
    def convert_scenes_format(scenes_torch):
        converted = []
        for scene_torch in scenes_torch:
            scene_converted = []
            for s in scene_torch:
                raw_field = s['raw']
                if isinstance(raw_field, torch.Tensor):
                    raw_np = raw_field.detach().cpu().numpy()
                else:
                    raw_np = np.asarray(raw_field)
                
                sensor_dict = {
                    'id': s['id'],
                    'type': s['type'],
                    'raw_rate': s['raw_rate'],
                    'raw': raw_np.astype(np.float32),
                    'meta': s.get('meta', {})
                }
                scene_converted.append(sensor_dict)
            converted.append(scene_converted)
        return converted
    
    # Convert train/val scenes
    scenes_train_for_preclassifier = convert_scenes_format(scenes_train)
    scenes_val_for_preclassifier = convert_scenes_format(scenes_val)
    
    # Train pre_classifier
    print("\n[Pre-classifier] Training on train set...")
    pre_classifier.train_on_scenes(scenes_train_for_preclassifier, device=device, 
                                    epochs=10, lr=1e-3, batch_size=64)
    
    # Validate pre_classifier
    print("\n[Pre-classifier] Validating on validation set...")
    preclassifier_metrics = pre_classifier.validate_on_scenes(
        scenes_val_for_preclassifier, device=device
    )
    
    print("\n[Pre-classifier] Validation Results:")
    for key, value in preclassifier_metrics.items():
        print(f"  {key:20s}: {value:.4f}")
    
    # Demo on sample scenes
    print("\n[Pre-classifier] Demo Classification on Sample Sensors:")
    demo_count = 0
    for scene_idx, scene in enumerate(scenes_val_for_preclassifier[:2]):
        print(f"\n  Scene {scene_idx}:")
        for sensor_idx, sensor in enumerate(scene[:5]):
            signal = sensor['raw']
            true_type = sensor['type']
            
            result = pre_classifier.classify_signal(signal)
            pred_type = result['type']
            confidence = result['confidence']
            
            match = "✓" if pred_type in true_type or true_type in pred_type else "✗"
            print(f"    {match} Sensor {sensor_idx}: True={true_type:15s} " +
                    f"Pred={pred_type:12s} Conf={confidence:.3f} Method={result['method']}")
            
            demo_count += 1
            if demo_count >= 15:
                break
        if demo_count >= 15:
            break
    
    print("\n[Pre-classifier] Summary:")
    print(f"  - Accuracy: {preclassifier_metrics.get('accuracy', 0.0):.4f}")
    print(f"  - Successfully classified {len(scenes_val_for_preclassifier)} validation scenes")
    print(f"  - Classification methods: heuristic + deep learning (hybrid)")
    
except ImportError as e:
    print(f"[Warning] Could not import pre_classifier: {e}")
    print("         Skipping pre-classifier integration.")
except Exception as e:
    print(f"[Error] Pre-classifier integration failed: {e}")
    import traceback
    traceback.print_exc()

# Quick hardware-aware tips
print("\nHardware tips:")
print(" - Use device='cuda' (Titan X) for heavy ops; enable mixed-precision (autocast used).")
print(" - DataLoader uses pin_memory + num_workers when GPU available.")
print(" - If memory allows, increase scene generation parallelism or batch size.")
print(" - For CPU-bound preprocessing consider increasing num_workers or pre-saving tensors.")
print(" - If training still slow: reduce emb_dim, reduce time_bins, or reduce dataset size for experiments.")
print("Done.")


[Pre-classifier] Initializing...

[Pre-classifier] Training on train set...
[Pre-classifier] Training on 1487 samples
[Pre-classifier] Epoch 1/10 loss=33.5838 acc=0.4607
[Pre-classifier] Epoch 2/10 loss=6.0001 acc=0.4499
[Pre-classifier] Epoch 3/10 loss=2.6903 acc=0.4250
[Pre-classifier] Epoch 4/10 loss=1.8158 acc=0.4412
[Pre-classifier] Epoch 5/10 loss=1.8108 acc=0.4196
[Pre-classifier] Epoch 6/10 loss=1.3327 acc=0.4412
[Pre-classifier] Epoch 7/10 loss=0.9606 acc=0.4909
[Pre-classifier] Epoch 8/10 loss=1.0587 acc=0.5783
[Pre-classifier] Epoch 9/10 loss=1.1752 acc=0.5837
[Pre-classifier] Epoch 10/10 loss=0.8788 acc=0.5629

[Pre-classifier] Validating on validation set...

[Pre-classifier] Validation Results:
  accuracy            : 0.6186
  1word_precision     : 0.6990
  1word_recall        : 1.0000
  1word_f1            : 0.8228
  2word_precision     : 1.0000
  2word_recall        : 0.0098
  2word_f1            : 0.0194
  bits_precision      : 0.4468
  bits_recall         : 0.8077
  